# 平均報酬における方策勾配の収束について

参考：
* [On the Global Convergence of Policy Gradient in Average Reward Markov Decision Processes](https://arxiv.org/abs/2403.06806)

割引無限ホライゾンでは方策勾配法が収束することを示すのは簡単です．
実際，

$$
\rho_\gamma^*-\rho_\gamma^{\pi_k} \leq O\left(\frac{1}{(1-\gamma)^5}\right)
$$

で収束します．これは平均報酬に残念ながらスケールしません．実際，$(1-\gamma)\to 1$に飛ばすと右辺が∞になってしまいます．

今回は平均報酬で最適方策を計算するための方策勾配の解析について考えます．

* **コメント**：結局割引率じゃなくてmixing coefficient $(1-\lambda)^{-1}$みたいな定数で抑えているっぽい．やってることはほぼ一緒かも．

表記：
* 平均報酬：$\rho^\pi=\lim _{N \rightarrow \infty} \frac{\mathbb{E}_\pi\left[\sum_{n=0}^{N-1} r^\pi\left(s_n\right)\right]}{N}$
  * $\rho^\pi=\sum_{s \in \mathcal{S}} d^\pi(s) r^\pi(s)$
* 平均ベルマン方程式：$\rho^\pi \mathbb{I}+v^\pi=r^\pi+\mathbb{P}^\pi v^\pi$,
  * この$v^\pi$は一意に定まりません．しかし，$\sum_{s \in \mathcal{S}} d^\pi(s) v^\pi(s)=0$なる制約を入れると一意になります
* $Q^\pi(s, a)=r(s, a)+\sum_{\substack{s^{\prime} \in \mathcal{S} \\ a^{\prime} \in \mathcal{A}}} \mathbb{P}\left(s^{\prime} \mid s, a\right) \pi\left(a^{\prime} \mid s^{\prime}\right) Q^\pi\left(s^{\prime}, a^{\prime}\right)-\rho^\pi$
* 方策勾配定理：$\frac{\partial \rho}{\partial \theta}=\sum_{s \in \mathcal{S}} d^\pi(s) \sum_{a \in \mathcal{A}} \frac{\partial \pi(s, a)}{\partial \theta} Q^\pi(s, a)$
  * 今回考える方策勾配：$\pi_{k+1}:=\operatorname{Proj}_{\Pi}\left[\pi_k+\left.\eta \frac{\partial \rho^\pi}{\partial \pi}\right|_{\pi=\pi_k}\right] \quad \forall k \geq 0$,


仮定：
* 任意の方策$\pi$について，その遷移行列$\mathbb{P}^\pi$がirreducibleかつaperiodicである．つまり，次を満たす定数$C_e<\infty$と$\lambda \in [0, 1)$が存在する：
任意の$k\in \mathbb{N}$と任意の$\pi \in \Pi$について，そのマルコフ連鎖がgeometrically ergodicである，つまり「定常分布に落ちる速度が指数的に早い」：
$$
\left\|\left(\mathbb{P}^\pi\right)^k-\mathbb{1}\left(d^\pi\right)^{\top}\right\|_{\infty} \leq C_e \lambda^k
$$

また，報酬は$[0, 1]$の範囲を考えます．


## 収束を示すための性質

[RL_PolicyGrad_convergence_rate](RL_PolicyGrad_convergence_rate.ipynb)での証明は，価値関数が方策に対してSmoothなことを利用し，Gradient dominanceを使って証明しました．

---

微分可能な関数$f: \mathcal{C}\to \mathbb{R}$は次を満たすときに$L$-smoothである，という：
$$
\|\nabla f(y)-\nabla f(x)\|_2 \leq L\|y-x\|_2 \quad \forall y, x \in \mathcal{C} .
$$

$L$-smoothな関数は次を満たします：
$$
|f(y)-f(x)-\langle\nabla f(x), y-x\rangle| \leq \frac{L}{2}\|y-x\|^2 \quad \forall y, x \in \mathcal{C},
$$
つまり，関数$cf$は$|c|L$-smoothです．ところで，割引ありの収益関数は$\frac{2\gamma |\mathcal{A}|}{(1-\gamma)^3}$-smoothであることが言えますが，平均報酬はどうでしょうか？

単純に$(1-\gamma)$をかけると，
$$
\lim_{\gamma \to 1}\frac{2\gamma |\mathcal{A}|}{(1-\gamma)^2}\to \infty
$$
smoothであり，smooth性が言えません．そこで，ベルマン方程式を使ってsmooth性を証明することにします．

$\Phi\in \mathbb{R}^{|\mathcal{S}|\times |\mathcal{S}|}$を，任意のベクトルを$\mathbb{I}$ベクトルに垂直な空間にユークリッドノルムの意味で射影する関数とします．このとき，次が成立します：

---

identity行列$I$と全ての要素が1のベクトル$\mathbb{I}$を考えます．どちらも次元が$|\mathcal{S}|$とします．このとき，$\Phi=\left(I-\frac{\mathbb{1 \mathbf { 1 } ^ { \top }}}{S}\right)$は垂直に射影する行列です．また，一意な価値観数は
$$
\begin{aligned}
&v_\phi^\pi=\Phi\left(r^\pi+\mathbb{P}^\pi v_\phi^\pi-\rho^\pi \mathbb{1}\right)\\
\text{もしくは }\quad&
v_\phi^\pi=\left(I-\Phi \mathbb{P}^\pi\right)^{-1} \Phi r^\pi
\end{aligned}
$$
で与えられます．

**証明**

$\mathbb{I}\in \mathbb{R}^{|S|}$と直行する平面$E$を考えます：

$$
E=\operatorname{span}\left\{\theta \in \mathbb{R}^{|\mathcal{S}|}: \theta^{\top} \mathbb{1}=0 .\right\}
$$

また適当なベクトル$v$の$E$への射影は

$$
v_\phi=\underset{u \in \mathbb{E}}{\arg \min }\|v-u\|_2
$$

で与えられます．この解析解は
$$
v_\phi=\left(I-\frac{\mathbb{1} \mathbb{1}^{\top}}{|\mathcal{S}|}\right) v
$$
で与えられます．ここで，任意の方策$\pi \in \Pi$について，ベクトル$r^\pi+\mathbb{P}^\pi v^\pi-\rho^\pi \mathbb{1}$の$E$への射影を考えてみましょう．
このような射影は一意に定まり，また，$r^\pi+\mathbb{P}^\pi v^\pi$を$E$に射影するのと等価です（$\rho^\pi \mathbb{I}$は$\Phi$のnull spaceにあるので）

$$
\begin{aligned}
\Phi\left(r^\pi-\rho^\pi \mathbb{1}+\mathbb{P}^\pi v\right) & =\left(r^\pi-\rho^\pi \mathbb{1}+\mathbb{P}^\pi v\right)-\left\langle\mathbb{1}, r^\pi-\rho^\pi \mathbb{I}+\mathbb{P}^\pi v\right\rangle \frac{\mathbb{1}}{|\mathcal{S}|} \\
& =\left(r^\pi+\mathbb{P}^\pi v\right)-\left\langle\mathbb{1}, r^\pi+\mathbb{P}^\pi v\right\rangle \frac{\mathbb{1}}{|\mathcal{S}|} \\
& =r^\pi-\left\langle r^\pi, \mathbb{1}\right\rangle \frac{\mathbb{1}}{|\mathcal{S}|}+\mathbb{P}^\pi v-\left\langle\mathbb{1}, \mathbb{P}^\pi v\right\rangle \frac{\mathbb{1}}{|\mathcal{S}|} \\
& =r^\pi-\left\langle r^\pi, \mathbb{1}\right\rangle \frac{\mathbb{1}}{|\mathcal{S}|}+\mathbb{P}^\pi v-\frac{\mathbb{1}}{|\mathcal{S}|}\left(\mathbb{1}^{\top} \mathbb{P}^\pi v\right) \\
& =\left(I-\frac{\mathbb{1} \mathbb{1}^{\top}}{|\mathcal{S}|}\right) r^\pi+\left(I-\frac{\mathbb{1} \mathbb{1}^{\top}}{|\mathcal{S}|}\right) \mathbb{P}^\pi v \\
& =\Phi\left[r^\pi+\mathbb{P}^\pi v\right] .
\end{aligned}
$$

ここで，$\pi$についての平均ベルマン方程式を考えてみましょう：
$\rho^\pi \mathbb{1}+v^\pi=r^\pi+\mathbb{P}^\pi v^\pi$

この方程式に$v^{\pi^{\top}} \mathbb{1}=0$の制約を追加すると，その解は一意に定まります．これを$v_\phi^\pi$と置きましょう．さらに，ベルマン方程式に射影を適用すると，
$$
\begin{aligned}
\Phi v_\phi^\pi+\Phi \rho^\pi \mathbb{1} & =\Phi r^\pi+\Phi \mathbb{P}^\pi v_\phi^\pi \\
\Longrightarrow \Phi v_\phi^\pi & =\Phi r^\pi+\Phi \mathbb{P}^\pi v_\phi^\pi \\
\Longrightarrow v_\phi^\pi & =\Phi\left[r^\pi+\mathbb{P}^\pi v_\phi^\pi\right]
\end{aligned}
$$
であることも言えます．最後の行は$v^{\pi^{\top}} \mathbb{1}=0 \Longrightarrow \Phi v^\pi=v^\pi$を使いました．よって，

$$
v_\phi^\pi=\left[I-\Phi \mathbb{P}^\pi\right]^{-1} \Phi r^\pi
$$

が成立します．逆行列の存在はノイマン級数展開から言えます．


---

さて，上の変形を使って得られた一意な価値関数$v_\phi^\pi$について，次のsmoothnessが言えます：

任意の方策について，$v^\pi_\phi$は$4\left(2 C_m^3 C_p^2 \kappa_r+C_m^2 C_p C_r\right)$-smoothになります．ここで各記号はいい感じの問題に依存した定数です．

**証明**

TODO: ちょっとめんどうなので飛ばす．Lemma 2を見よう．頑張って$\mathbb{P}^\pi$が$\pi$についてsmoothであることを示すっぽい．

---

また，$\rho^\pi$がLipschitz連続であることが言えます．